# Social Graph Analysis Group Project

### Grandma’s Cookbook, Lookbook: An Analysis of Ingredients Association With Ethnic Cuisines

#### Group Members:
* Nick Potts
* Kristi Conry
* Jacob Lebowitz
* Anthony Bryson


### Possible Goals :)

1) Predict popular cuisine's based on what ingediants you have <br>
2) Essential ingrediants if you like a favorite cuisine <br>
3) Ingrediant recommendation system ( if you lke 3 types of cuisine find top similar ingrediants ) 

### Just some questions for our data :)

1) Which cuisine's are most similar based on ingediants? and why?

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import json
import collections
import re
import pickle

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [2]:
df = pd.read_json('data/train.json')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39774 entries, 0 to 39773
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           39774 non-null  int64 
 1   cuisine      39774 non-null  object
 2   ingredients  39774 non-null  object
dtypes: int64(1), object(2)
memory usage: 932.3+ KB


In [4]:
df.head(5)

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


In [5]:
df['ingredients_sentence'] = df['ingredients'].apply(lambda x: ' '.join(x))

In [6]:
df.head()

,id,cuisine,ingredients,ingredients_sentence
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olives grape tomatoes ga...
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomatoes ground...
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggs pepper salt mayonaise cooking oil green c...
3,22213,indian,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallots cornflour cayenne pepper...


In [7]:
ps = PorterStemmer()

def clean_ingredients(text):    
    #Remove punctuations
    text=re.sub(r'[^\w\s]','',text)
    
    #Remove Digits
    text=re.sub(r"(\d)", "", text)
    
    #Remove content inside paranthesis
    text=re.sub(r'\([^)]*\)', '', text)
    
    #Remove Brand Name
    text=re.sub(u'\w*\u2122', '', text)
    
    #Convert to lowercase
    text=text.lower()
    
    #Remove Stop Words
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    text=' '.join(filtered_sentence)
        
    
    #Porter Stemmer Algorithm
    words = word_tokenize(text)
    word_ps=[]
    for w in words:
        word_ps.append(ps.stem(w))
    text=' '.join(word_ps)
    
    return text
    
    

In [8]:
df['ingredient_cleaned'] = df['ingredients_sentence'].apply(clean_ingredients)

In [9]:
df['ingredient_list_cleaned'] = df['ingredients'].apply(lambda x: list(map(clean_ingredients, x)))

In [19]:
df.head()

,id,cuisine,ingredients,ingredients_sentence,ingredient_cleaned,ingredient_list_cleaned
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes...",romaine lettuce black olives grape tomatoes ga...,romain lettuc black oliv grape tomato garlic p...,"[romain lettuc, black oliv, grape tomato, garl..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g...",plain flour ground pepper salt tomatoes ground...,plain flour ground pepper salt tomato ground b...,"[plain flour, ground pepper, salt, tomato, gro..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g...",eggs pepper salt mayonaise cooking oil green c...,egg pepper salt mayonais cook oil green chili ...,"[egg, pepper, salt, mayonais, cook oil, green ..."
3,22213,indian,"[water, vegetable oil, wheat, salt]",water vegetable oil wheat salt,water veget oil wheat salt,"[water, veget oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe...",black pepper shallots cornflour cayenne pepper...,black pepper shallot cornflour cayenn pepper o...,"[black pepper, shallot, cornflour, cayenn pepp..."


In [20]:
df.to_csv('data/cleaned_data.csv', index=False)